# Bibliotecas e Configurações

1. Requer:
    * `./data/GlobalLandTemperaturesByCity.csv`;
    * `./data/co2_data.csv` ou `./out/co2_data.csv`.

<br />

2. É necessário possuir o **Java SE Development Kit 11** (JDK) instalado;
    * Outras versões apresentam problemas;
    * Possivel baixar o `jdk-11.0.16` através desse *[link](https://gist.github.com/wavezhang/ba8425f24a968ec9b2a8619d7c2d86a6)*.

<br />

3. Além disso, é necessário definir a variável de ambiente `JAVA_HOME` cujo valor é o diretório de instalação do JDK;
    * Exemplo: `C:\Progra~1\Java\jdk-11.0.16`;
    * Em que `Progra~1` é, na verdade, a pasta `Program Files`.

<br />

4. Definir também as variáveis de ambiente:
    * `PYSPARK_DRIVER_PYTHON` = `jupyter`;
    * `PYSPARK_PYTHON` = `python`.

<br />

5. Além disso tudo, é necessário o Spark + Hadoop para salvar arquivos;
    1. Assim sendo, efetue o download da versão 3.3.1 do [Spark](https://spark.apache.org/downloads.html);
    2. Extraia os arquivos para um diretório desejado do computador (local de instalação);
    3. Dentro desse diretório de instalação do Spark crie o diretório do Haddop: `<spark_path>\hadoop\bin`;
    4. Efetuar o download da versão 3.3.1 do [Hadoop](https://github.com/kontext-tech/winutils), baixar o repositório como ZIP e extrair todos os arquivos da versão desejada na pasta `bin` recém criada.

<br />

6. Definir as variáveis de ambiente:
    * `SPARK_HOME`: diretório de instalação do Spark;
    * `HADOOP_HOME` = `%SPARK_HOME%\hadoop\bin`.

<br />

7. Adicionar os caminhos dessa variáveis (`SPARK_HOME` E `HADOOP_HOME`) ao Path;

<br />

8. Será necessário reiniciar o computador após definir as variáveis de ambiente e alterar o Path.
    * Há abaixo uma verificação para atestar se é possível ler todas as variáveis necessárias.

In [2]:
import os

environ_vars = [
    'JAVA_HOME',
    'PYSPARK_DRIVER_PYTHON',
    'PYSPARK_PYTHON',
    'SPARK_HOME',
    'HADOOP_HOME'
]

for var in environ_vars:
    if not os.environ.get(var):
        print(f'AVISO: Variável {var} não encontrada.')

In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import expr
from pyspark.sql.utils import AnalysisException

# Leitura

In [3]:
spark = SparkSession.builder.getOrCreate()

tpr_data = spark.read.options(header='True').csv('./data/GlobalLandTemperaturesByCity.csv')

try: 
    co2_data = spark.read.options(header='True').csv('./out/co2_data.csv')
except AnalysisException:
    co2_data = spark.read.options(header='True').csv('./data/co2_data.csv')

In [4]:
tpr_data.show(5)

+----------+------------------+-----------------------------+-----+-------+--------+---------+
|        dt|AverageTemperature|AverageTemperatureUncertainty| City|Country|Latitude|Longitude|
+----------+------------------+-----------------------------+-----+-------+--------+---------+
|1743-11-01|             6.068|           1.7369999999999999|Århus|Denmark|  57.05N|   10.33E|
|1743-12-01|              null|                         null|Århus|Denmark|  57.05N|   10.33E|
|1744-01-01|              null|                         null|Århus|Denmark|  57.05N|   10.33E|
|1744-02-01|              null|                         null|Århus|Denmark|  57.05N|   10.33E|
|1744-03-01|              null|                         null|Århus|Denmark|  57.05N|   10.33E|
+----------+------------------+-----------------------------+-----+-------+--------+---------+
only showing top 5 rows



*Dataset* de temperaturas:

* `dt` (string): data da amostra;
* `AverageTemperature` (float): temperatura média medida;
* `AverageTemperatureUncertainty` (float): incerteza associada a temperatura medida;
* `City` (string): cidade de origem da amostra;
* `Country` (string): país de origem da amostra;
* `Latitude` (string): graus de latitude norte da amostra;
* `Longitude` (string):  graus de longitude leste da amostra;

In [5]:
co2_data.show(5)

+----------+------+------+-----------------+
|     Times|LatDim|LonDim|            value|
+----------+------+------+-----------------+
|1850-01-01|     0|     0|288.1340637207031|
|1850-01-01|     0|     1|288.1340637207031|
|1850-01-01|     0|     2|288.1340637207031|
|1850-01-01|     0|     3|288.1340637207031|
|1850-01-01|     0|     4|288.1340637207031|
+----------+------+------+-----------------+
only showing top 5 rows



*Dataset* de temperaturas:

* `Times` (string): data da amostra;
* `LatDim` (int): graus de latitude norte da amostra;
* `LonDim` (int): graus de longitude leste da amostra;
* `value` (float): concentração de $CO_2$ em ppm;

# Tratamento

* Define uma mesma data de início para ambos os datasets
* Realiza correção de tipos de dados e padroniza colunas

In [6]:
# Padroniza datasets iniciando em 1850-01-01
tpr_data = tpr_data[tpr_data['dt'] >= '1850-01-01']

In [7]:
# Padroniza nomes de colunas entre datasets

trp_columns_rename = [
    ('dt', 't_date'),
    ('AverageTemperature', 't_temperature'),
    ('AverageTemperatureUncertainty', 't_temperature_unc'),
    ('City', 't_city'),
    ('Country', 't_country'),
    ('Latitude', 't_latitude'),
    ('Longitude', 't_longitude'),
]

co2_columns_rename = [
    ('Times', 'c_date'),
    ('LatDim', 'c_latitude'),
    ('LonDim', 'c_longitude'),
    ('value', 'c_co2'),
]

for old, new in trp_columns_rename:
    tpr_data = tpr_data.withColumnRenamed(old, new)

for old, new in co2_columns_rename:
    co2_data = co2_data.withColumnRenamed(old, new)

In [8]:
# Retira 'N' e 'E' do final dos valores de 
# latitude/longitude e converte para float

to_strip = ['t_latitude', 't_longitude']

for col in to_strip:
    tpr_data = tpr_data.withColumn(
        col,
        expr(f"float(substring({col}, 1, length({col})-1))")
    )

In [9]:
tpr_data.show(5)

+----------+--------------------+-----------------+------+---------+----------+-----------+
|    t_date|       t_temperature|t_temperature_unc|t_city|t_country|t_latitude|t_longitude|
+----------+--------------------+-----------------+------+---------+----------+-----------+
|1850-01-01|              -5.265|             1.82| Århus|  Denmark|     57.05|      10.33|
|1850-02-01|               1.859|            1.641| Århus|  Denmark|     57.05|      10.33|
|1850-03-01|0.031999999999999806|            3.167| Århus|  Denmark|     57.05|      10.33|
|1850-04-01|  5.7639999999999985|            1.903| Århus|  Denmark|     57.05|      10.33|
|1850-05-01|              11.037|            0.586| Århus|  Denmark|     57.05|      10.33|
+----------+--------------------+-----------------+------+---------+----------+-----------+
only showing top 5 rows



In [10]:
co2_data.show(5)

+----------+----------+-----------+-----------------+
|    c_date|c_latitude|c_longitude|            c_co2|
+----------+----------+-----------+-----------------+
|1850-01-01|         0|          0|288.1340637207031|
|1850-01-01|         0|          1|288.1340637207031|
|1850-01-01|         0|          2|288.1340637207031|
|1850-01-01|         0|          3|288.1340637207031|
|1850-01-01|         0|          4|288.1340637207031|
+----------+----------+-----------+-----------------+
only showing top 5 rows



# Integralização

* Une datasets pelo critério de distância mínima

    1. Para cada ponto de tempetura, obtém a sua data e encontra todos os pontos de $CO_2$ nessa mesma data;
    2. Calcula a distância entre o ponto de temperatura e todos os outros pontos de $CO_2$ anteriormente selecionados;
    3. Escolhe aquele de mínima distância como o ponto equivalente entre os *datasets*.

<br />

* Cálculo da distância
    * ✔️ **Abordagem 1:** plana — $d(P_1, P_2) = \sqrt{(x_1-x_2)^2 + (y_1-y_2)^2}$
    * ❓ **Abordagem 2:** esférica — Faz sentido? É necessária? Como fazer?

<br />

* Sobre a implementação
    * Não encontrei uma forma direta e elegante para fazer a integração apenas através da API do PySpark, então optei por apelar ao SQL 🙏

In [11]:
# Recorte ínfimo dos datasets apenas para validação

tpr_data.limit(int(1e3)).createOrReplaceTempView("tpr_data")
co2_data.limit(int(1e5)).createOrReplaceTempView("co2_data")

In [12]:
# Raw query de integralização

cross_data = spark.sql(
    """
    SELECT *
    FROM (
        -- Ordenação crescente das distâncias,
        -- orientado a data, latitude e longitude
        SELECT
            *,
            ROW_NUMBER() OVER (
                PARTITION BY
                    t_date,
                    t_latitude,
                    t_longitude
                ORDER BY distance ASC
            ) AS row_num
        FROM (
            -- CROSS JOIN para cálculo de distâncias entre
            -- todos os pontos dos datasets em uma mesma data
            SELECT
                *,
                SQRT(POW(t_latitude - c_latitude, 2) + POW(t_longitude - c_longitude, 2)) AS distance
            FROM tpr_data AS T
            CROSS JOIN co2_data AS C
            ON t_date = c_date
        )
    )
    WHERE row_num = 1; -- Escolha dos registros com menor distância
    """
)

In [13]:
cross_data.show(10)

+----------+-------------+-----------------+------+---------+----------+-----------+----------+----------+-----------+-----------------+-------------------+-------+
|    t_date|t_temperature|t_temperature_unc|t_city|t_country|t_latitude|t_longitude|    c_date|c_latitude|c_longitude|            c_co2|           distance|row_num|
+----------+-------------+-----------------+------+---------+----------+-----------+----------+----------+-----------+-----------------+-------------------+-------+
|1850-01-01|       -5.265|             1.82| Århus|  Denmark|     57.05|      10.33|1850-01-01|        57|         10|285.5076904296875|0.33376619564095866|      1|
|1850-02-01|        1.859|            1.641| Århus|  Denmark|     57.05|      10.33|1850-02-01|        57|         10|286.4536437988281|0.33376619564095866|      1|
+----------+-------------+-----------------+------+---------+----------+-----------+----------+----------+-----------+-----------------+-------------------+-------+



In [16]:
cross_data.write.format('csv').option('header', True).mode('overwrite').save('./tmp')